In [2]:
from datasets import DatasetDict, load_from_disk, load_metric
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)
import pandas as pd
import numpy as np  #2차원, 3차원 배열 표현이 쉽다.
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter
import json

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터셋 가져오기

In [7]:
datasets = load_from_disk('../../dataset/train_dataset')
datasets_test = load_from_disk('../../dataset/test_dataset')

In [8]:
train_df = datasets['train'].to_pandas()
vaild_df = datasets['validation'].to_pandas()
test_df = datasets_test['validation'].to_pandas()

# 커리큘럼 러닝

### 난이도 하 데이터 가져오기
전체 train_dataset에 대해 inference

In [10]:
# inference 결과
with open('../../models/05-03-35_train)pred/nbest_predictions.json') as f:
    js = json.loads(f.read())

In [11]:
#prob 가장 높은 값만 가져오기
train_answers={}
for k,v in js.items():
    max_prob_answer = v[0]
    for answer in v:
        if max_prob_answer['probability'] < answer['probability']:
            max_prob_answer = answer
    train_answers[k] = answer


In [12]:
len(train_answers)

3952

In [13]:
q_ids = []    #dataset의 id
answer_start_idx = []    #실제 정답 시작 index
answer_end_idx = []      #실제 정답 끝 index
pred_start_idx = []      #inference 정답 시작 index
pred_end_idx = []        #inference 정답 끝 index
for k,v in train_answers.items():
    q_ids.append(k)
    answer_start_idx.append(v['answer_offsets'][0])
    answer_end_idx.append(v['answer_offsets'][1])
    pred_start_idx.append(v['offsets'][0])
    pred_end_idx.append(v['offsets'][1])

In [14]:
#L2 loss 계산
import math
def metrics(asi,psi,aei,pei):
    return math.sqrt(math.pow(asi-psi,2)+math.pow(aei-pei,2))
loss = []
for i, ids in enumerate(q_ids):
    loss.append(metrics(answer_start_idx[i], pred_start_idx[i],answer_end_idx[i],pred_end_idx[i]))
print(len(loss))


3952


In [35]:
# 데이터가 1/3을 유지하게 하위 1/3을 저장
bi_loss=[1 if i < 8 else 0 for i in loss]
train_ids = [q_ids[i] for i in range(len(loss)) if bi_loss[i] == 1]

In [36]:
len(train_ids)

1291

In [37]:
train_ids= {q_ids[i] : bi_loss[i] for i in range(len(loss))}

In [14]:
with open('Curriculum_Learning.json', "w", encoding="utf-8") as writer:
    writer.write(
        json.dumps(train_ids, indent=4, ensure_ascii=False) + "\n"
    )

In [15]:
with open('Curriculum_Learning.json') as f:
    js3 = json.loads(f.read())

### 난이도 중 계산
난이도 하와 같은 계산을 반복

In [39]:
# inference 결과2
with open('../../models/05-05-58_1차 ㅜㅎ/nbest_predictions.json') as f:
    js2 = json.loads(f.read())
    
#prob 가장 높은 값만 가져오기
train_answers2={}
for k,v in js2.items():
    max_prob_answer = v[0]
    for answer in v:
        if max_prob_answer['probability'] < answer['probability']:
            max_prob_answer = answer
    train_answers2[k] = answer

q_ids2 = []
answer_start_idx2 = []
answer_end_idx2 = []
pred_start_idx2 = []
pred_end_idx2 = []
for k,v in train_answers2.items():
    q_ids2.append(k)
    answer_start_idx2.append(v['answer_offsets'][0])
    answer_end_idx2.append(v['answer_offsets'][1])
    pred_start_idx2.append(v['offsets'][0])
    pred_end_idx2.append(v['offsets'][1])

loss2 = []
for i, ids in enumerate(q_ids2):
    loss2.append(metrics(answer_start_idx2[i], pred_start_idx2[i],answer_end_idx2[i],pred_end_idx2[i]))
    


In [46]:
bi_loss2=[1 if i < 25 else 0 for i in loss2]
for i in range(len(bi_loss2)):
    if bi_loss[i]==1:
        bi_loss2[i]=0
train_ids2_list = [q_ids2[i] for i in range(len(loss2)) if bi_loss2[i] == 1]
train_ids2= {q_ids2[i] : bi_loss2[i] for i in range(len(loss2))}
print(len(train_ids2_list))
len(train_ids2)

1330


3952

In [19]:
with open('Curriculum_Learning2.json', "w", encoding="utf-8") as writer:
    writer.write(
        json.dumps(train_ids2, indent=4, ensure_ascii=False) + "\n"
    )

### 난이도 상 계산
난이도 하와 중을 제외하면 나머지가 난이도 상

In [49]:
bi_loss3 = [1 for i in range(len(loss2))]
for i in range(len(bi_loss2)):
    if bi_loss[i]==1:
        bi_loss3[i]=0
for i in range(len(bi_loss2)):
    if bi_loss2[i]==1:
        bi_loss3[i]=0     
print(sum(bi_loss3))

1331


In [21]:
train_ids3_list = [q_ids2[i] for i in range(len(loss2)) if bi_loss3[i] == 1]
train_ids3= {q_ids2[i] : bi_loss3[i] for i in range(len(loss2))}

In [22]:
with open('Curriculum_Learning3.json', "w", encoding="utf-8") as writer:
    writer.write(
        json.dumps(train_ids2, indent=4, ensure_ascii=False) + "\n"
    )